In [1]:
import csv
import numpy as np
import pandas as pd
import scipy.optimize
import scipy.stats
import colorcet as cc

import bokeh.io
import bokeh.plotting
import bokeh.palettes
import bokeh.models

bokeh.io.output_notebook()

Loading BokehJS ...

In [2]:
ic_df = pd.read_csv('../data/ion_chromatography/ion_chromatography_tidy.csv')
ic_df.head()

,Sample ID,timepoint,Nitrite (mM),Nitrate (mM),Acetate (mM),row,column,ace,pca,strain,Time (hours)
0,B1,T0,0.0777,10.3053,0.0552,B,1,0,0 µM PCA,C. portucalensis MBL,0.0
1,C1,T0,0.0455,10.5099,0.0328,C,1,0,0 µM PCA,C. portucalensis MBL,0.0
2,D1,T0,0.0695,10.8882,0.0187,D,1,0,0 µM PCA,C. portucalensis MBL,0.0
3,B4,T0,0.0775,10.3049,0.0543,B,4,0,200 µM PCAox,C. portucalensis MBL,0.0
4,C4,T0,0.0844,10.6758,0.0196,C,4,0,200 µM PCAox,C. portucalensis MBL,0.0


In [3]:
abio_ic_df = ic_df.loc[ic_df['row'] == 'E']
bio_ic_df = ic_df.loc[ic_df['row'] != 'E']

In [4]:
bio_ic_df

,Sample ID,timepoint,Nitrite (mM),Nitrate (mM),Acetate (mM),row,column,ace,pca,strain,Time (hours)
0,B1,T0,0.0777,10.3053,0.0552,B,1,0,0 µM PCA,C. portucalensis MBL,0.0
1,C1,T0,0.0455,10.5099,0.0328,C,1,0,0 µM PCA,C. portucalensis MBL,0.0
2,D1,T0,0.0695,10.8882,0.0187,D,1,0,0 µM PCA,C. portucalensis MBL,0.0
3,B4,T0,0.0775,10.3049,0.0543,B,4,0,200 µM PCAox,C. portucalensis MBL,0.0
4,C4,T0,0.0844,10.6758,0.0196,C,4,0,200 µM PCAox,C. portucalensis MBL,0.0
...,...,...,...,...,...,...,...,...,...,...,...
103,C6,T3,11.4206,0.0078,46.9181,C,6,50,200 µM PCAox,C. portucalensis MBL,53.0
104,D6,T3,11.2461,0.0145,47.5036,D,6,50,200 µM PCAox,C. portucalensis MBL,53.0
105,B9,T3,11.2677,0.0154,47.0770,B,9,50,200 µM PCAred,C. portucalensis MBL,53.0
106,C9,T3,11.4955,0.0086,48.1258,C,9,50,200 µM PCAred,C. portucalensis MBL,53.0


In [5]:
grouped = bio_ic_df.groupby(['Time (hours)', 'strain', 'pca', 'ace'])

times = []
strains = []
pcas = []
aces = []

no3_means = []
no2_means = []
ace_means = []

no3_standard_errors = []
no2_standard_errors = []
ace_standard_errors = []

for g in grouped:
    
    times.append(g[0][0])
    strains.append(g[0][1])
    pcas.append(g[0][2])
    aces.append(g[0][3])
    
    no3_means.append(np.mean(g[1]['Nitrate (mM)']))
    no3_standard_errors.append(np.std(g[1]['Nitrate (mM)'])/np.sqrt(3))
    
    no2_means.append(np.mean(g[1]['Nitrite (mM)']))
    no2_standard_errors.append(np.std(g[1]['Nitrite (mM)'])/np.sqrt(3))
    
    ace_means.append(np.mean(g[1]['Acetate (mM)']))
    ace_standard_errors.append(np.std(g[1]['Acetate (mM)'])/np.sqrt(3))

no3_stat_df = pd.DataFrame.from_dict({'strain': strains,
                                      'Time (hours)': times,
                                      'pca': pcas,
                                      'ace': aces,
                                      'mid': no3_means,
                                      'se': no3_standard_errors,})

no2_stat_df = pd.DataFrame.from_dict({'strain': strains,
                                      'Time (hours)': times,
                                      'pca': pcas,
                                      'ace': aces,
                                      'mid': no2_means,
                                      'se': no2_standard_errors,})

ace_stat_df = pd.DataFrame.from_dict({'strain': strains,
                                      'Time (hours)': times,
                                      'pca': pcas,
                                      'ace': aces,
                                      'mid': ace_means,
                                      'se': ace_standard_errors,})

no3_stat_df['low'] = no3_stat_df['mid'] - 1.96 * no3_stat_df['se']
no3_stat_df['high'] = no3_stat_df['mid'] + 1.96 * no3_stat_df['se']

no2_stat_df['low'] = no2_stat_df['mid'] - 1.96 * no2_stat_df['se']
no2_stat_df['high'] = no2_stat_df['mid'] + 1.96 * no2_stat_df['se']

ace_stat_df['low'] = ace_stat_df['mid'] - 1.96 * ace_stat_df['se']
ace_stat_df['high'] = ace_stat_df['mid'] + 1.96 * ace_stat_df['se']

In [6]:
no3_stat_df

,strain,Time (hours),pca,ace,mid,se,low,high
0,C. portucalensis MBL,0.0,0 µM PCA,0,10.567800,0.139409,10.294558,10.841042
1,C. portucalensis MBL,0.0,0 µM PCA,10,10.227733,0.096935,10.037740,10.417727
2,C. portucalensis MBL,0.0,0 µM PCA,50,10.456767,0.064266,10.330805,10.582729
3,C. portucalensis MBL,0.0,200 µM PCAox,0,10.488133,0.087441,10.316750,10.659517
4,C. portucalensis MBL,0.0,200 µM PCAox,10,10.273867,0.030638,10.213816,10.333918
5,C. portucalensis MBL,0.0,200 µM PCAox,50,10.371233,0.056511,10.260472,10.481995
6,C. portucalensis MBL,0.0,200 µM PCAred,0,10.385700,0.074868,10.238960,10.532440
7,C. portucalensis MBL,0.0,200 µM PCAred,10,10.322467,0.061947,10.201051,10.443883
8,C. portucalensis MBL,0.0,200 µM PCAred,50,10.392467,0.060903,10.273097,10.511836
9,C. portucalensis MBL,8.0,0 µM PCA,0,10.541967,0.120661,10.305472,10.778461


In [7]:
no2_stat_df

,strain,Time (hours),pca,ace,mid,se,low,high
0,C. portucalensis MBL,0.0,0 µM PCA,0,0.064233,0.007888,0.048772,0.079694
1,C. portucalensis MBL,0.0,0 µM PCA,10,0.163100,0.002076,0.159031,0.167169
2,C. portucalensis MBL,0.0,0 µM PCA,50,0.168767,0.003950,0.161025,0.176508
3,C. portucalensis MBL,0.0,200 µM PCAox,0,0.080100,0.001768,0.076634,0.083566
4,C. portucalensis MBL,0.0,200 µM PCAox,10,0.164933,0.001559,0.161877,0.167990
5,C. portucalensis MBL,0.0,200 µM PCAox,50,0.167200,0.004290,0.158792,0.175608
6,C. portucalensis MBL,0.0,200 µM PCAred,0,0.040433,0.005653,0.029353,0.051514
7,C. portucalensis MBL,0.0,200 µM PCAred,10,0.175467,0.008805,0.158209,0.192724
8,C. portucalensis MBL,0.0,200 µM PCAred,50,0.166367,0.005726,0.155143,0.177590
9,C. portucalensis MBL,8.0,0 µM PCA,0,0.238900,0.003342,0.232350,0.245450


In [8]:
def plot_ic_bokeh(no3_df, no2_df):
    
    no3_grouped = no3_df.groupby('ace')
    no2_grouped = no2_df.groupby('ace')

    plots = []

    for g in zip(no3_grouped, no2_grouped):
        
        no3_g, no2_g = g
        
        ace = no3_g[0]
        no3_mini_df = no3_g[1]
        no2_mini_df = no2_g[1]

        fig = bokeh.plotting.figure(width=600, 
                                    height=400, 
                                    title=f'{ace} mM acetate',
                                    x_axis_label = 'Time (hrs)',
                                    y_axis_label = f'Nitrate or Nitrite (mM)')

        no3_mini_group = no3_mini_df.groupby('pca')
        no2_mini_group = no2_mini_df.groupby('pca')

        legend_items = []

        for i, mg in enumerate(zip(no3_mini_group, no2_mini_group)):
            
            no3_mg, no2_mg = mg
            
            pca = no3_mg[0]
            
            no3_mdf = no3_mg[1]
            no2_mdf = no2_mg[1]

            color = bokeh.palettes.Colorblind3[i]

            c = fig.circle(no3_mdf['Time (hours)'], no3_mdf['mid'], color=color, size=7, alpha=1, line_color='black')
            l = fig.line(no3_mdf['Time (hours)'], no3_mdf['mid'], color=color, line_width=2)
            
            s = fig.square(no2_mdf['Time (hours)'], no2_mdf['mid'], color=color, size=7, alpha=1, line_color='black')
            l2 = fig.line(no2_mdf['Time (hours)'], no2_mdf['mid'], color=color, line_width=2)

            legend_items.append((f"{pca} (NO3)", [l, c,]))
            legend_items.append((f"{pca} (NO2)", [l2, s]))

            no3_xs = no3_mdf['Time (hours)'].values
            no3_lows = no3_mdf['low'].values
            no3_highs = no3_mdf['high'].values

            no3_err_xs = []
            no3_err_ys = []

            for x, l, h in zip(no3_xs, no3_lows, no3_highs):
                no3_err_xs.append((x, x))
                no3_err_ys.append((l, h))

            no3_error = fig.multi_line(no3_err_xs, no3_err_ys, color='grey', line_width=1.5, alpha=1)
            
            no2_xs = no2_mdf['Time (hours)'].values
            no2_lows = no2_mdf['low'].values
            no2_highs = no2_mdf['high'].values

            no2_err_xs = []
            no2_err_ys = []

            for x, l, h in zip(no2_xs, no2_lows, no2_highs):
                no2_err_xs.append((x, x))
                no2_err_ys.append((l, h))

            no2_error = fig.multi_line(no2_err_xs, no2_err_ys, color='grey', line_width=1.5, alpha=1)


        legend = bokeh.models.Legend(items=legend_items)
        legend.click_policy = "hide"

        fig.add_layout(legend, 'right')
        
        fig.y_range = bokeh.models.Range1d(-0.5, 12)

        fig.legend.label_text_font_size = '14pt'
        fig.title.text_font_size = "14pt"

        fig.yaxis.axis_label_text_font_size = '14pt'
        fig.xaxis.axis_label_text_font_size = '14pt'
        fig.yaxis.major_label_text_font_size = '14pt'
        fig.xaxis.major_label_text_font_size = '14pt'

        fig.output_backend = 'svg'

        plots.append(fig)
        
    return(plots)

In [9]:
ic_plots = plot_ic_bokeh(no3_stat_df, no2_stat_df)

In [10]:
bokeh.io.show(ic_plots[2])

In [11]:
bokeh.io.export_svgs(ic_plots[0], filename='./plots/figure18_0mM_acetate.svg')

['./plots/figure18_0mM_acetate.svg']

In [12]:
bokeh.io.export_svgs(ic_plots[1], filename='./plots/figure18_10mM_acetate.svg')

['./plots/figure18_10mM_acetate.svg']

In [13]:
bokeh.io.export_svgs(ic_plots[2], filename='./plots/figure18_50mM_acetate.svg')

['./plots/figure18_50mM_acetate.svg']

In [14]:
df_for_rates = bio_ic_df.loc[(bio_ic_df['Time (hours)'] < 10) & (bio_ic_df['ace'] == 0)]
df_for_rates

,Sample ID,timepoint,Nitrite (mM),Nitrate (mM),Acetate (mM),row,column,ace,pca,strain,Time (hours)
0,B1,T0,0.0777,10.3053,0.0552,B,1,0,0 µM PCA,C. portucalensis MBL,0.0
1,C1,T0,0.0455,10.5099,0.0328,C,1,0,0 µM PCA,C. portucalensis MBL,0.0
2,D1,T0,0.0695,10.8882,0.0187,D,1,0,0 µM PCA,C. portucalensis MBL,0.0
3,B4,T0,0.0775,10.3049,0.0543,B,4,0,200 µM PCAox,C. portucalensis MBL,0.0
4,C4,T0,0.0844,10.6758,0.0196,C,4,0,200 µM PCAox,C. portucalensis MBL,0.0
5,D4,T0,0.0784,10.4837,0.0296,D,4,0,200 µM PCAox,C. portucalensis MBL,0.0
6,B7,T0,0.0279,10.2029,0.0233,B,7,0,200 µM PCAred,C. portucalensis MBL,0.0
7,C7,T0,0.0416,10.4898,0.0195,C,7,0,200 µM PCAred,C. portucalensis MBL,0.0
8,D7,T0,0.0518,10.4644,0.0187,D,7,0,200 µM PCAred,C. portucalensis MBL,0.0
27,B1,T1,0.2334,10.2574,0.0178,B,1,0,0 µM PCA,C. portucalensis MBL,8.0


In [15]:
grouped = df_for_rates.groupby('pca')

for g in grouped:
    
    hrs = g[1]['Time (hours)'].values
    no3 = g[1]['Nitrate (mM)'].values
    no2 = g[1]['Nitrite (mM)'].values
    
    #no3 reduction
    no3_s, no3_i, no3_r, no3_p, no3_e = scipy.stats.linregress(hrs, no3)
    
    #no2 production
    no2_s, no2_i, no2_r, no2_p, no2_e = scipy.stats.linregress(hrs, no2)
    
    print(f"""
    In the {g[0]} condition:
    Nitrate reduction rate was {no3_s:.3f} +/- {1.96 * no3_e:.3f} mM/hr
    Nitrite production rate was {no2_s:.3f} +/- {1.96 * no2_e:.3f} mM/hr
    """)
    


    In the 0 µM PCA condition:
    Nitrate reduction rate was -0.003 +/- 0.055 mM/hr
    Nitrite production rate was 0.022 +/- 0.003 mM/hr
    

    In the 200 µM PCAox condition:
    Nitrate reduction rate was -0.035 +/- 0.035 mM/hr
    Nitrite production rate was 0.058 +/- 0.002 mM/hr
    

    In the 200 µM PCAred condition:
    Nitrate reduction rate was -0.131 +/- 0.049 mM/hr
    Nitrite production rate was 0.147 +/- 0.044 mM/hr
    


In [16]:
%load_ext watermark
%watermark -v -p numpy,scipy,pandas,bokeh,jupyterlab,holoviews,colorcet

Python implementation: CPython
Python version       : 3.9.15
IPython version      : 8.7.0

numpy     : 1.23.4
scipy     : 1.9.3
pandas    : 1.5.2
bokeh     : 2.3.3
jupyterlab: 3.5.0
holoviews : 1.14.9
colorcet  : 3.0.1

